In [52]:
using DataFrames;
using JuMP;
using Chemostat;
using Chemostat.Tools;
using Plots;
plotly();

## Loading Model

In [53]:
S, mets, rxns = Chemostat.v1.Models.EColi.load_ecoli_v1();
println("S $(size(S)), mets $(size(mets)) rxns $(size(rxns))");

S (72, 96), mets (72, 2) rxns (96, 6)


In [67]:
find((row) -> contains(uppercase(row[:id]), "ATPM"), eachrow(rxns))

1-element Array{Int64,1}:
 11

In [68]:
rxns[11,:]

,id,lb,ub,ap,an,t
1,ATPM,0.0,1000.0,0.012151,0.012151,0.0


In [75]:
mets[S[:,11].nzind,:]

,id,c
1,adp,0.0
2,atp,0.0
3,h2o,0.0
4,h,0.0
5,pi,0.0


In [73]:
ATPMmets = S[:,11]

72-element SparseVector{Float64,Int64} with 5 stored entries:
  [13]  =  1.0
  [17]  =  -1.0
  [41]  =  -1.0
  [43]  =  1.0
  [60]  =  1.0

In [74]:
ATPMmets.nzind

5-element Array{Int64,1}:
 13
 17
 41
 43
 60

## Test at ξmax

In [65]:
ξmax = 3e6/;#3e6
md = Chemostat.v1.Models.EColi.DEFAULT_MAINTINANCE_DEMAND_FLUX_VALUE; #8e-5;
cost = Chemostat.v1.Models.EColi.DEFAULT_ϕ;#1e-4
fbares = Chemostat.v1.FBA.fba_chemostat_v1(S, mets, rxns, ξmax; ϕub = cost, man_demand_flux_value = md);
println();
println("Results")
println("ϕ: $(getvalue(fbares.ϕ))");
println("Obj: $(getvalue(fbares.obj))");
println();
print("Non zero transport fluxes: ")
println(filter((x) -> x[2] != 0 ,map((x,y) -> (x,y), rxns[:id][find((row) -> row[:t] > 0, eachrow(rxns))],
    Chemostat.v1.FBA.flux.(fbares, rxns[:id][find((row) -> row[:t] > 0, eachrow(rxns))]))))

Academic license - for non-commercial use only

Results
ϕ: 0.0001
Obj: 1.8973071653473414e-5

Non zero transport fluxes: Tuple{String,Float64}[("ACt2r", -3.86151e-5), ("CO2t", -0.000438226), ("GLCpts", 0.000226151), ("H2Ot", -0.000582261), ("O2t", 0.000415366)]


In [55]:
ξrange = Tools.logrange(0.001,50, ξmax);
println("ξrange length: $(length(ξrange)), start $(start(ξrange)), last $(last(ξrange))")

ξrange length: 255, start 1, last 3.0e6


## Solving

In [56]:
fbaress = [Chemostat.v1.FBA.fba_chemostat_v1(S,mets,rxns,ξ; man_demand_flux_value = md, ϕub = cost) for ξ in ξrange];

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic lice

## Substrate and Waste

In [57]:
Sid = "glc-D[e]";
Sindx = 35;
Stid = "GLCpts";
Stinx = 49;#50
Wid = "ac[e]";
Windx = 6;
Wtid = "ACt2r";
Wtindx = 6;

## S vs ξ

In [58]:
spdata = Chemostat.v1.FBA.s.(fbaress,Sid,Stid);
sp = plot((ξrange, spdata), xaxis = :log, lw = 2)

## W vs ξ

In [59]:
wpdata = Chemostat.v1.FBA.s.(fbaress,Wid,Wtid);
wp = plot((ξrange,wpdata), xaxis = :log, lw = 2)

## μ vs ξ

In [60]:
τ = 0.000003;
mupdata = Chemostat.v1.FBA.μ.(fbaress, wpdata, τ);
mup = plot((ξrange, mupdata), xaxis = :log, lw =2)

## X vs ξ

In [61]:
Xpdata = Chemostat.v1.FBA.X.(fbaress,mupdata,1)
Xp = plot((ξrange, Xpdata), xaxis = :log, lw = 2)

## X vs D (τ > 0)

In [62]:
X_vs_Dpdata = plot((mupdata, Xpdata),lw = 2)

## X vs D (τ = 0)

In [63]:
τ = 0.0;
mupdatant = Chemostat.v1.FBA.μ.(fbaress, wpdata, τ);
Xpdatant = Chemostat.v1.FBA.X.(fbaress,mupdatant,1);
X_vs_Dpdata = plot((mupdatant, Xpdatant), lw = 2)